# Esta Notebook sirve para probar la generacion de respuestas de nuestros modelos entre usuarios de JPC y FDT

Ejecutar en Google Colab o en si se quiere ejecutar de manera local se requiere Python 3.6 y tensorflow 1.15

Se puede conseguir corriendo:

    conda create --name GPT2GPU python=3.6 jupyter -y
    conda activate GPT2GPU
    conda install --name GPT2GPU tensorflow-gpu=1.15 --channel conda-forge -y

ó para correrlo con CPU

    conda create --name GPT2CPU python=3.6 jupyter -y
    conda activate GPT2CPU
    conda install --name GPT2CPU tensorflow=1.15 --channel conda-forge -y


## Configuracion antes de ejecutar

se pueden cambiar algunos valores aqui y volver a correr toda la notebook para que entrene y guarde otro modelo

In [1]:
# Nombre del Modelo que vas a generar despues del finetunning, Ejemplo: FDT ó JPC
MODEL_NAME = "FDT"   

# COLAB or LOCAL
ENVIRONMENT = "LOCAL" 

# TEXTO DE ENTRADA
INPUT_TEXT= "Que se vayan todos!"


### NO TOCAR
MODEL_DIRS='models'
if ENVIRONMENT=="LOCAL":
    MODEL_DIRS ='../' + MODEL_DIRS
    CHECKPOINT_DIR=MODEL_DIRS
else:
    CHECKPOINT_DIR="checkpoint"

#  Empieza la notebook

In [2]:
# Si esto no dice que tenes la version 1.15 de tensorflow no va a funcionar!
#!pip show tensorflow

In [3]:
if ENVIRONMENT=="COLAB":
    %tensorflow_version 1.x
    from google.colab import files
else:
    !pip install -q tensorflow-plot==0.3.0
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory usa tanto una Nvidia T4 GPU o una Nvidia K80 GPU. La T4 es ligeramente mas rapida que la K80

Podes verificar en que estas corriendo usando esta celda

In [4]:
#!nvidia-smi

## Montamos Google Drive

En caso de usar modelos directo de Google Drive, dejarlos en la raiz

In [5]:
if ENVIRONMENT=="COLAB":
    gpt2.mount_gdrive()
    gpt2.copy_checkpoint_from_gdrive(run_name=MODEL_NAME)

## Cargamos el modelo en memoria

In [6]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=MODEL_NAME,checkpoint_dir=MODEL_DIRS)

Loading checkpoint ../models/FDT/model-2000
INFO:tensorflow:Restoring parameters from ../models/FDT/model-2000


## Generamos el resultado

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [7]:
responses = gpt2.generate(sess,
              checkpoint_dir=CHECKPOINT_DIR,
              run_name=MODEL_NAME,
              length=250,
              temperature=.9,
              prefix="[POST] " + INPUT_TEXT + " \n[RESPONSE]",
              nsamples=5,
              batch_size=5,
              truncate="<|endoftext|>",
              return_as_list=True,
              include_prefix=False
              )

## Vemos El resultado

In [8]:
responses

[' @uberYbarra En lugar deuda y mierda yo, creo que llegó a una persona que miente !!! Que es una operación ! LOS COSAS !!!\n',
 ' @_Trompeta Sí me acordarlo por los colores de lo que decías, se sostiene un nombre de puertas y por los trabajadores de @C5N. Acordarlo se vayan las exercisemen. Igual lo quiero.\n',
 ' @_LauraDiMarco @Agos102005 Nadie es la sincera, que Alverso se va a poner la imagen desde que ML difundir la cenación.\n',
 ' @Winston_Dunhill #YoHoyMiroABaby ¿y vos? ¿y vos dijo la verdad?\n',
 ' @FernandezAnibal Era hacercelo posible para esto... https://t.co/7htZ26NpHD\n']